<a href="https://colab.research.google.com/github/kKn00077/techit-diary-final-project/blob/main/ai_model/ko_emotion_dataset_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 라이브러리 설치
# !pip install torch transformers sentencepiece datasets evaluate gcsfs
# !pip install keras --upgrade

In [29]:
# 감정 라벨 분류 데이터
# https://huggingface.co/datasets/m2af/ko-emotion-dataset
from datasets import load_dataset
dataset = load_dataset("m2af/ko-emotion-dataset")
print(dataset)
print(dataset['train'][0])

DatasetDict({
    train: Dataset({
        features: ['created_date', 'source', 'context', 'annotation', '__index_level_0__'],
        num_rows: 1039
    })
})
{'created_date': '2024-02-03', 'source': 'X', 'context': '보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이 있어서 울었다네요', 'annotation': "[{'annotator': 'A', 'emotion': '기쁨_만족감', 'gender': 'F'}, {'annotator': 'B', 'emotion': '기쁨_만족감', 'gender': 'M'}, {'annotator': 'C', 'emotion': '기쁨_감동', 'gender': 'F'}, {'annotator': 'E', 'emotion': '기쁨_감동', 'gender': 'F'}, {'annotator': 'D', 'emotion': '기쁨_만족감', 'gender': 'M'}]", '__index_level_0__': 0}


In [30]:
from collections import Counter
import json
import pandas as pd

# 데이터셋 변환 작업
transformed_data = []

for entry in dataset['train']:
    # annotation 필드를 JSON으로 파싱
    annotations = json.loads(entry['annotation'].replace("'", '"'))
    # 모든 emotion 값을 리스트로 추출
    emotions = [annotation['emotion'] for annotation in annotations]
    # 최빈값 계산
    most_common_emotion = Counter(emotions).most_common(1)[0][0]
    # 필요한 데이터만 추출
    transformed_data.append({
        'created_date': entry['created_date'],
        'context': entry['context'],
        'emotion': most_common_emotion
    })

# 변환된 데이터를 DataFrame으로 생성
df = pd.DataFrame(transformed_data)

# emotion 열에서 '_' 이후의 부분 제거
df['emotion'] = df['emotion'].str.split('_').str[0]

# 결과 출력
display(df)
display(df.value_counts('emotion'))

,created_date,context,emotion
0,2024-02-03,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,기쁨
1,2024-02-07,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,기쁨
2,2024-02-08,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,기쁨
3,2024-02-09,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,슬픔
4,2024-02-10,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,기쁨
...,...,...,...
1034,2024-02-21,축협이랑 감독도 문제인데 이강인은 할말이 없네 멱살 잡았다고 주먹을 날리냐..물론 ...,미움(상대방)
1035,2024-02-21,애초에 이게 진실이든 협회가 그냥 시선 돌리는거이든 협회는 뭔 생각으로 진실이라고 ...,미움(상대방)
1036,2024-02-21,문뜩 10년째 인성논란없이 쭉 정상에 서있는 페이커가 새삼 대단하게 느껴지네...,기쁨
1037,2024-02-21,이게 이렇게까지 욕 먹을 일인가 싶다.. 물론 잘못이 있지만 본인들은 잘못하나 안저...,싫어함(상태)


,count
emotion,
기쁨,248
슬픔,179
욕망,116
싫어함(상태),109
두려움,100
분노,92
미움(상대방),80
사랑,76
수치심,29


In [ ]:
# 레이블 수정

# 싫어함(상태)를 싫음으로 변경
df['emotion'] = df['emotion'].replace('싫어함(상태)', '싫음')

# 수치심과 두려움을 두려움으로 변경
df['emotion'] = df['emotion'].replace(['수치심', '두려움'], '두려움')

# 분노와 미움(상대방)을 분노로 변경
df['emotion'] = df['emotion'].replace(['분노', '미움(상대방)'], '미움')

# 중립 삭제
df = df[df['emotion'] != '중립']

# 새로운 값의 분포 확인
display(df['emotion'].value_counts())

,count
emotion,
기쁨,248
슬픔,179
미움,172
두려움,129
욕망,116
싫음,109
사랑,76


In [ ]:
# 전처리 : 숫자, 이모티콘, 특수문자 제거
import re

# 숫자, 이모티콘, 특수문자 제거
def preprocess(text):
    # 이모티콘, 유니코드 범위 및 특수 문자를 제거하는 정규식 (숫자는 제외)
    emoji_pattern = re.compile(
        r"["
        r"\d+"                    # 숫자 제거
        r"\U0001F600-\U0001F64F"  # 이모티콘 범위 (유니코드)
        r"\U0001F300-\U0001F5FF"  # 기호 및 아이콘
        r"\U0001F680-\U0001F6FF"  # 운송 및 기계 관련 유니코드
        r"\U0001F700-\U0001F77F"  # 기타 유니코드 기호
        r"\U0001F780-\U0001F7FF"  # 추가 유니코드 기호
        r"\U0001F800-\U0001F8FF"  # 유니코드 기술 문자
        r"\U0001F900-\U0001F9FF"  # 제스처와 감정 관련 이모티콘
        r"\U0001FA00-\U0001FA6F"  # 이모티콘 확장
        r"\U0001FB00-\U0001FBFF"  # 추가 확장 이모티콘
        r"\u2600-\u26FF"          # 기호 및 도형 (예: ♠, ♣)
        r"\u2700-\u27BF"          # 기타 기호 (예: ✨, ✋)
        r"]+",
        flags=re.UNICODE
    )

    # 정규식을 이용해 매칭되는 문자 제거
    return emoji_pattern.sub(r'', str(text))

# 모든 텍스트에 대해 전처리 적용
df = df.copy()  # 경고 방지: 원본 데이터 복사
df['processed_context'] = df['context'].apply(preprocess)

In [ ]:
# 레이블 인코딩 (기쁨 -> 0, 사랑-> 1 등)
label_mapping = { '기쁨': 0, '사랑' : 1, '슬픔': 2, '미움': 3, '두려움': 4, '욕망': 5, '싫음': 6 }

# emotion 열을 숫자로 매핑
df.loc[:, 'labels'] = df['emotion'].map(label_mapping)

# 결과 확인
display(df.head(), df.shape)

,created_date,context,emotion,processed_context,labels
0,2024-02-03,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,기쁨,보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이...,0
1,2024-02-07,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,기쁨,어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스...,0
2,2024-02-08,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,기쁨,미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드...,0
3,2024-02-09,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,슬픔,요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ,2
4,2024-02-10,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,기쁨,크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은...,0


(1029, 5)

In [ ]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
from evaluate import load
from transformers import EarlyStoppingCallback


# 데이터 준비
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# 모델 이름 설정 및 토크나이저 로드
model_name = "beomi/KcELECTRA-base-v2022"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)

# 데이터 전처리 함수 정의
def preprocess_function(examples):
    return tokenizer(examples['processed_context'], truncation=True, padding='max_length', max_length=128)

# 데이터셋 변환
train_dataset = Dataset.from_pandas(train_df[['processed_context', 'labels']])
test_dataset = Dataset.from_pandas(test_df[['processed_context', 'labels']])

# 토크나이즈 적용
train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# 불필요한 컬럼 제거
train_dataset = train_dataset.remove_columns(['processed_context'])
test_dataset = test_dataset.remove_columns(['processed_context'])

# 데이터셋 형식 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# 정확도 계산을 위한 metric 로드
metric = load('accuracy')

# compute_metrics 함수 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model='accuracy',
    greater_is_better=True
)

# EarlyStopping 설정
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=3,  # 검증 성능이 개선되지 않아도 기다릴 최대 에폭 수
    early_stopping_threshold=0.001  # 개선으로 간주할 최소 변화량
)


# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]  # EarlyStoppingCallback 추가
)

# 모델 학습
trainer.train()

# 베스트 모델 저장
trainer.save_model('best_model')
tokenizer.save_pretrained('./best_model')

print("Model training completed. Best model saved in 'best_model' directory.")


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/309 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,1.809200,1.682923,0.430421
2,1.582100,1.464160,0.491909
3,1.344500,1.308762,0.521036
4,1.042400,1.254378,0.543689
5,0.881100,1.131226,0.588997
6,0.621100,1.110963,0.608414
7,0.452700,1.077753,0.618123
8,0.342500,1.085678,0.592233
9,0.270300,1.100656,0.605178
10,0.220700,1.117432,0.601942


Model training completed. Best model saved in 'best_model' directory.


In [ ]:
# 모델 및 토크나이저 불러오기
model_path = "./best_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 모델을 평가 모드로 설정
model.eval()

# 입력 데이터 생성
test_sentences = [
                  '야옹이가 강아지와 알콩달콩하는게 너무 귀여워.',
                  '야옹이가 강아지와 알콩달콩하는게 너무 사랑스러워.',
                  '야옹이가 강아지와 알콩달콩하는게 너무 귀여워. 하지만 모두 죽겠지',
                  '야옹이가 강아지와 알콩달콩하는게 너무 귀여워. 하지만 모두 죽겠지, 그 아이들이 사라진다면 화가 날 것 같아!',
                  '반려동물을 싫어하는 사람들과는 함께 있고 싶지 않아',
                  '야옹이가 강아지와 알콩달콩하는게 너무 귀여워. 이 아이들과 함께 영원히 살고 싶어',
                  '야옹이랑 강아지가 그만 싸웠으면 좋겠어. 모두 사이좋게 지내길 바래'
                  ]  # 예시 문장

inputs = tokenizer(
    test_sentences,
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=128
    )

# 예측 수행
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_labels = torch.argmax(logits, dim=-1).tolist()  # 모든 문장의 예측값 리스트로 반환

label_decoding = { 0 : '기쁨', 1 : '사랑', 2 : '슬픔', 3 : '분노', 4 : '두려움', 5 : '욕망', 6 : '싫음' }
for sentence, label in zip(test_sentences, predicted_labels):
    print(f"{sentence} ===> {label_decoding[label]}")

야옹이가 강아지와 알콩달콩하는게 너무 귀여워. ===> 기쁨
야옹이가 강아지와 알콩달콩하는게 너무 사랑스러워. ===> 기쁨
야옹이가 강아지와 알콩달콩하는게 너무 귀여워. 하지만 모두 죽겠지 ===> 사랑
야옹이가 강아지와 알콩달콩하는게 너무 귀여워. 하지만 모두 죽겠지, 그 아이들이 사라진다면 화가 날 것 같아! ===> 슬픔
반려동물을 싫어하는 사람들과는 함께 있고 싶지 않아 ===> 싫음
야옹이가 강아지와 알콩달콩하는게 너무 귀여워. 이 아이들과 함께 영원히 살고 싶어 ===> 사랑
야옹이랑 강아지가 그만 싸웠으면 좋겠어. 모두 사이좋게 지내길 바래 ===> 욕망


In [ ]:
# 파인튜닝한 모델 압축하여 로컬컴퓨터에 저장
!zip -r best_model.zip ./best_model

from google.colab import files
files.download('best_model.zip')

  adding: best_model/ (stored 0%)
  adding: best_model/model.safetensors (deflated 7%)
  adding: best_model/tokenizer.json (deflated 70%)
  adding: best_model/training_args.bin (deflated 51%)
  adding: best_model/config.json (deflated 56%)
  adding: best_model/vocab.txt (deflated 52%)
  adding: best_model/tokenizer_config.json (deflated 75%)
  adding: best_model/special_tokens_map.json (deflated 42%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>